# Data loading 
Next cells are used to load the data we need.

In [1]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [7]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

URM_train_ICM_all_IALS = sps.vstack([URM_all*(1-0.51668), ICM_all.T*0.51668])
recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
recommender_IALS.fit(n_factors = 601, regularization = 0.8715, iterations=83)

Recommender_Base_Class: URM Detected 2 (0.01 %) cold users.


In [13]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

URM_train_ICM_all_RP3 = sps.vstack([URM_all, ICM_all.T])
recommender_rp3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta.fit(topK=330, alpha = 0.32049, beta=0.2480, implicit = False) #beta=0.19798705532101593 da provare
recommender_rp3Beta.URM_train = URM_train

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.


In [9]:
from KNN.UserKNNCFRecommender import UserKNNCFRecommender

URM_train_ICM_all_USER = sps.vstack([URM_all*(1-0.4662), ICM_all.T*0.4662])
recommender_USER = UserKNNCFRecommender(URM_train_ICM_all_USER)
recommender_USER.fit(shrink=158, topK=158, feature_weighting = "TF-IDF")

UserKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
Similarity column 27947 ( 100 % ), 4682.23 column/sec, elapsed time 0.10 min


In [14]:
from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])

hybrid = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_rp3Beta], 2)
hybrid.fit([0.100,0.874])
#evaluator_validation.evaluateRecommender(hybrid)

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 2


In [15]:
test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [16]:
user_id = test_users['user_id']
recommendations = hybrid.recommend(user_id,cutoff = 10)


In [17]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
print(len(recommendations))

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

test_users
test_users.to_csv('submission.csv', index=False)


7944
